In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-jupyter.bash notebook
```

This notebook makes heavy use of the following python libraries.
- `pandas`: from the python computing stack (http://pandas.pydata.org/pandas-docs/stable/)
- `geojson`: standard JSON representation of geographic data (http://geojson.org/)
- `folium`: (https://github.com/python-visualization/folium) python bridge to leaflet (http://leafletjs.com/)

### Pick a user to work with ###

In [1]:
pwd

'/var/emission/e-mission-server'

In [1]:
import emission.core.get_database as edb
import pandas as pd
all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
all_users

storage not configured, falling back to sample, default configuration
Connecting to database URL localhost


,user_email,uuid
0,patgendre,a2ad418f-74ff-4d60-8ea0-9ccf9dd5de9b
1,mahieu,bb5eb9ad-4589-4aa7-a4cb-f1c2037dbcd4
2,maria.tebar@cerema.fr,2aa06302-bb3b-48d9-805e-d5ae6209bfca
3,test_user_1,c1010f8c-b089-4d77-84ed-d19a3a4656c8


In [3]:
test_user_id_2 = all_users.iloc[2].uuid
test_user_id_7 = all_users.iloc[7].uuid

In [4]:
from uuid import UUID

In [2]:
test_user_id = all_users.iloc[2].uuid # replace with UUID from above

If you want to work across multiple users, just do the same thing again

In [11]:
print("tre")

tre


### Preferred access technique

The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later. The timeseries is conceptually a set of streams, one for each of the types, primarily indexed by time. So you can query for all entries of a particular time within a specified time range.

In [3]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [6]:
print(test_user_id_2)
ts_2 = esta.TimeSeries.get_time_series(test_user_id_2)
ts_7 = esta.TimeSeries.get_time_series(test_user_id_7)

91bc554a-4228-4afd-b2ca-be8eb7e33760


#### Accessing entries directly

In [4]:
# Get all cleaned trips for the first user
ts = esta.TimeSeries.get_time_series(test_user_id)
entry_it = ts.find_entries(["segmentation/raw_trip"], time_query=None)

All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [8]:
# recherche des tracés d'iti user 
entry_rawtrips = ts.find_entries(["analysis/cleaned_trip"], time_query=None)
for ct in entry_rawtrips:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    print ("jour-heure ",cte.data.start_local_dt, cte.data.duration)

=== Trip: {"coordinates": [-1.6720301, 48.10444], "type": "Point"} -> {"coordinates": [-1.6733677, 48.1043067], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 3, 'day': 27, 'hour': 19, 'minute': 28, 'second': 54, 'weekday': 2, 'timezone': 'Europe/Paris'}) 100.21199989318848
=== Trip: {"coordinates": [3.0619455, 50.6283401], "type": "Point"} -> {"coordinates": [3.0654961, 50.6270542], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 4, 'day': 19, 'hour': 12, 'minute': 44, 'second': 1, 'weekday': 4, 'timezone': 'Europe/Paris'}) 909.8414974212646
=== Trip: {"coordinates": [3.0654961, 50.6270542], "type": "Point"} -> {"coordinates": [3.0558097, 50.6307208], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 4, 'day': 19, 'hour': 17, 'minute': 15, 'second': 54, 'weekday': 4, 'timezone': 'Europe/Paris'}) 271.9594702720642
=== Trip: {"coordinates": [3.0685092, 50.6244877], "type": "Point"} -> {"coordinates": [3.0702096, 50.6272042], "type": "Point"}
j

In [9]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    print ("jour-heure ",cte.data.start_local_dt, cte.data.duration)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [10]:
print (cte.data)

Cleanedtrip({'source': 'DwellSegmentationTimeFilter', 'end_ts': 1556195003.195, 'end_local_dt': {'year': 2019, 'month': 4, 'day': 25, 'hour': 14, 'minute': 23, 'second': 23, 'weekday': 3, 'timezone': 'Europe/Paris'}, 'end_fmt_time': '2019-04-25T14:23:23.195000+02:00', 'end_loc': {'type': 'Point', 'coordinates': [3.0654589, 50.6271102]}, 'raw_trip': ObjectId('5cc1b36c4e276dea15c700b3'), 'start_ts': 1556194389.264322, 'start_local_dt': {'year': 2019, 'month': 4, 'day': 25, 'hour': 14, 'minute': 13, 'second': 9, 'weekday': 3, 'timezone': 'Europe/Paris'}, 'start_fmt_time': '2019-04-25T14:13:09.264322+02:00', 'start_loc': {'type': 'Point', 'coordinates': [3.0702096, 50.6272042]}, 'duration': 613.9306778907776, 'distance': 442.25737408352654, 'start_place': ObjectId('5cc1b36d4e276dea15c700f2'), 'end_place': ObjectId('5cc1b36d4e276dea15c700f3')})


In [11]:
# Get all cleaned trips for the second user
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)

In [12]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

=== Trip: {"coordinates": [-1.6720301, 48.10444], "type": "Point"} -> {"coordinates": [-1.6733677, 48.1043067], "type": "Point"}
  --- Section: {"coordinates": [-1.6720301, 48.10444], "type": "Point"} -> {"coordinates": [-1.6733677, 48.1043067], "type": "Point"}  on  MotionTypes.ON_FOOT
=== Trip: {"coordinates": [3.0619455, 50.6283401], "type": "Point"} -> {"coordinates": [3.0654961, 50.6270542], "type": "Point"}
  --- Section: {"coordinates": [3.0619455, 50.6283401], "type": "Point"} -> {"coordinates": [3.0654961, 50.6270542], "type": "Point"}  on  MotionTypes.ON_FOOT
=== Trip: {"coordinates": [3.0654961, 50.6270542], "type": "Point"} -> {"coordinates": [3.0558097, 50.6307208], "type": "Point"}
  --- Section: {"coordinates": [3.0654961, 50.6270542], "type": "Point"} -> {"coordinates": [3.0558097, 50.6307208], "type": "Point"}  on  MotionTypes.UNKNOWN
=== Trip: {"coordinates": [3.0685092, 50.6244877], "type": "Point"} -> {"coordinates": [3.0702096, 50.6272042], "type": "Point"}
  --- S

In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
entry_it_2 = ts_2.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(list(entry_it)), test_user_id_2, len(list(entry_it_2))))

#### Accessing a dataframe

In [13]:
# Get all cleaned trips for the first user
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
sm_df = ts.get_data_df("analysis/smoothing", time_query=None)

In [14]:
len(ct_df)

5

In [15]:
ct_df.columns

Index(['_id', 'distance', 'duration', 'end_fmt_time', 'end_loc',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_month', 'end_local_dt_second', 'end_local_dt_timezone',
       'end_local_dt_weekday', 'end_local_dt_year', 'end_place', 'end_ts',
       'metadata_write_ts', 'raw_trip', 'source', 'start_fmt_time',
       'start_loc', 'start_local_dt_day', 'start_local_dt_hour',
       'start_local_dt_minute', 'start_local_dt_month',
       'start_local_dt_second', 'start_local_dt_timezone',
       'start_local_dt_weekday', 'start_local_dt_year', 'start_place',
       'start_ts', 'user_id'],
      dtype='object')

In [16]:
ct_df[["start_loc", "end_loc", "start_fmt_time", "end_fmt_time"]]

,start_loc,end_loc,start_fmt_time,end_fmt_time
0,"{'type': 'Point', 'coordinates': [-1.6720301, ...","{'type': 'Point', 'coordinates': [-1.6733677, ...",2019-03-27T19:28:54.788000+01:00,2019-03-27T19:30:35+01:00
1,"{'type': 'Point', 'coordinates': [3.0619455, 5...","{'type': 'Point', 'coordinates': [3.0654961, 5...",2019-04-19T12:44:01.458503+02:00,2019-04-19T12:59:11.300000+02:00
2,"{'type': 'Point', 'coordinates': [3.0654961, 5...","{'type': 'Point', 'coordinates': [3.0558097, 5...",2019-04-19T17:15:54.152530+02:00,2019-04-19T17:20:26.112000+02:00
3,"{'type': 'Point', 'coordinates': [3.0685092, 5...","{'type': 'Point', 'coordinates': [3.0702096, 5...",2019-04-25T12:12:41.083000+02:00,2019-04-25T12:15:51.942000+02:00
4,"{'type': 'Point', 'coordinates': [3.0702096, 5...","{'type': 'Point', 'coordinates': [3.0654589, 5...",2019-04-25T14:13:09.264322+02:00,2019-04-25T14:23:23.195000+02:00


In [34]:
# Get all cleaned trips for the second user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
ct_df_2[["start_loc", "end_loc", "start_ts", "end_ts"]]

,start_loc,end_loc,start_ts,end_ts
0,"{'type': 'Point', 'coordinates': [5.3732571, 4...","{'type': 'Point', 'coordinates': [5.470528, 43...",1.548694e+09,1.548696e+09
1,"{'type': 'Point', 'coordinates': [5.470528, 43...","{'type': 'Point', 'coordinates': [5.4705162, 4...",1.548696e+09,1.548698e+09
2,"{'type': 'Point', 'coordinates': [5.4657783, 4...","{'type': 'Point', 'coordinates': [5.4703689, 4...",1.548874e+09,1.548876e+09
3,"{'type': 'Point', 'coordinates': [5.4660641, 4...","{'type': 'Point', 'coordinates': [5.3736241, 4...",1.548919e+09,1.548921e+09


In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(ct_df), test_user_id_2, len(ct_df_2)))

In [ ]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [ ]:
len(cs_df)

In [ ]:
cs_df.columns

In [ ]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [21]:
import emission.core.get_database as edb

In [22]:
edb.get_timeseries_db().find_one()

{'_id': ObjectId('5c49e7944e276d05c81a293b'),
 'user_id': '',
 'metadata': {'key': 'stats/server_api_time',
  'platform': 'server',
  'write_ts': 1548347284.757146,
  'time_zone': 'America/Los_Angeles',
  'write_local_dt': {'year': 2019,
   'month': 1,
   'day': 24,
   'hour': 8,
   'minute': 28,
   'second': 4,
   'weekday': 3,
   'timezone': 'America/Los_Angeles'},
  'write_fmt_time': '2019-01-24T08:28:04.757146-08:00'},
 'data': {'name': 'GET_/',
  'ts': 1548347284.75363,
  'reading': 0.02602386474609375}}

In [23]:
edb.get_timeseries_db().distinct("metadata.key")

['stats/server_api_time',
 'background/location',
 'background/filtered_location',
 'statemachine/transition',
 'background/motion_activity',
 'stats/pipeline_time',
 'stats/client_nav_event',
 'background/battery',
 'stats/client_time',
 'config/consent',
 'stats/client_error',
 'stats/server_api_error']

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [24]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

['segmentation/raw_trip',
 'segmentation/raw_place',
 'segmentation/raw_section',
 'segmentation/raw_stop',
 'analysis/smoothing',
 'analysis/recreated_location',
 'analysis/cleaned_section',
 'analysis/cleaned_stop',
 'analysis/cleaned_place',
 'analysis/cleaned_trip',
 'segmentation/raw_untracked',
 'analysis/cleaned_untracked',
 'inference/prediction',
 'analysis/inferred_section']

In [26]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id_2, "metadata.key": "analysis/cleaned_trip"}).count()

4

In particular, you can use this to access entries that are not in the timeseries

In [27]:
edb.get_uuid_db().distinct("uuid")

[UUID('40267d4c-5fe9-423c-b18d-cde3d09c68c2'),
 UUID('d0cbf849-f9b1-4ba1-83be-1a148ec56a1e'),
 UUID('91bc554a-4228-4afd-b2ca-be8eb7e33760')]

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [ ]:
ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [ ]:
import emission.storage.decorations.timeline as esdl

In [ ]:
trip_start_end_fuzz = 10 # seconds
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts - trip_start_end_fuzz, ct_df.iloc[-1].end_ts + trip_start_end_fuzz)

In [ ]:
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
# The timeline is an iterator, so after it is consumed, it is empty
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.

### Plot a processed trip or set of trips ###

In [17]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

analysis.debug.conf.json not configured, falling back to sample, default configuration


In [18]:
first_trip_for_user = ct_df.iloc[2]
first_trip_start_ts = first_trip_for_user.start_ts
first_trip_end_ts = first_trip_for_user.end_ts
trip_start_end_fuzz = 10 # seconds
trips_geojson_list = gfc.get_geojson_for_ts(test_user_id, first_trip_start_ts-trip_start_end_fuzz, ct_df.iloc[-1].end_ts+trip_start_end_fuzz)

In [19]:
len(trips_geojson_list)

3

In [20]:
map_list = lo.get_maps_for_geojson_trip_list(trips_geojson_list)

start_place 5cbefade4e276d76770075b7
end_place 5cc1b36c4e276dea15c700f0
section 5cc1b36c4e276dea15c700c0
start_place 5cc1b36d4e276dea15c700f1
end_place 5cc1b36d4e276dea15c700f2
section 5cc1b36c4e276dea15c700ce
start_place 5cc1b36d4e276dea15c700f2
end_place 5cc1b36d4e276dea15c700f3
section 5cc1b36c4e276dea15c700d9


In [21]:
len(map_list)

3

In [22]:
map_list[2]

In [36]:
map_list[0]

In [32]:
import branca.element as bre

In [33]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, m in enumerate(map_list[:6]):
    fig.add_subplot(nrows,ncols,i+1).add_child(m)
fig

In [34]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, map in enumerate(map_list[-6:]):
    fig.add_subplot(nrows,ncols,i+1).add_child(map)
fig

## Can you do better? ##


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [25]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

Locations go from all = 11 -> filtered = 10 -> resampled = 5


In [26]:
all_locs[["_id", "latitude", "longitude", "fmt_time"]]

,_id,latitude,longitude,fmt_time
0,5cbece743070fe2fdf38d3c2,48.104440,-1.672030,2019-03-27T19:28:54.788000+01:00
1,5cbece743070fe2fdf38d3bc,48.104440,-1.672030,2019-03-27T19:28:54.788000+01:00
2,5cbece743070fe2fdf38d3ca,48.104429,-1.671971,2019-03-27T19:29:01+01:00
3,5cbece743070fe2fdf38d3d0,48.104437,-1.671968,2019-03-27T19:29:05.125000+01:00
4,5cbece743070fe2fdf38d3d6,48.104419,-1.671956,2019-03-27T19:29:10.146000+01:00
5,5cbece743070fe2fdf38d3de,48.104459,-1.671972,2019-03-27T19:29:17+01:00
6,5cbece743070fe2fdf38d3e4,48.104472,-1.672020,2019-03-27T19:29:22+01:00
7,5cbece743070fe2fdf38d3ea,48.104472,-1.672100,2019-03-27T19:29:27+01:00
8,5cbece743070fe2fdf38d3f0,48.104443,-1.672207,2019-03-27T19:29:32+01:00
9,5cbece743070fe2fdf38d3fe,48.104408,-1.672752,2019-03-27T19:30:06+01:00


### Get the raw motion activity, in case you want to do different segmentation ###

In [27]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [28]:
all_activity.columns

Index(['_id', 'confidence', 'fmt_time', 'local_dt_day', 'local_dt_hour',
       'local_dt_minute', 'local_dt_month', 'local_dt_second',
       'local_dt_timezone', 'local_dt_weekday', 'local_dt_year',
       'metadata_write_ts', 'ts', 'type', 'user_id'],
      dtype='object')

In [29]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

Found 0 walking entries, 12 on_foot entries
Found 0 motorized entries


### Plot the location points ###

In [30]:
map_list = lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                gfc.get_feature_list_from_df(filtered_locs),
                                                gfc.get_feature_list_from_df(resampled_locs)])

Found 11 features from 11 points
Found 10 features from 10 points
Found 5 features from 5 points
Points are  [[48.10444, -1.6720301], [48.10444, -1.6720301], [48.1044294, -1.6719714], [48.1044367, -1.6719678], [48.1044186, -1.6719558]] ...
Linestring is [[48.10444, -1.6720301], [48.10444, -1.6720301], [48.1044294, -1.6719714], [48.1044367, -1.6719678], [48.1044186, -1.6719558]] ...
Points are  [[48.10444, -1.6720301], [48.1044294, -1.6719714], [48.1044367, -1.6719678], [48.1044186, -1.6719558], [48.1044593, -1.6719724]] ...
Linestring is [[48.10444, -1.6720301], [48.1044294, -1.6719714], [48.1044367, -1.6719678], [48.1044186, -1.6719558], [48.1044593, -1.6719724]] ...
Points are  [[48.10444, -1.6720301], [48.10447164423999, -1.672065108001709], [48.10441910771754, -1.6725722090722983], [48.10434219550308, -1.6731507830367516], [48.1043067, -1.6733677]] ...
Linestring is [[48.10444, -1.6720301], [48.10447164423999, -1.672065108001709], [48.10441910771754, -1.6725722090722983], [48.10434

In [35]:
fig = bre.Figure()
for i, map in enumerate(map_list):
    fig.add_subplot(1,3,i+1).add_child(map)
fig